# Task 4: Creating an Interactive Chat Interface

In [8]:
# %pip install streamlit
# %pip install gradio

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
import os 
import sys
sys.path.append(os.path.abspath("../src"))
import warnings
warnings.filterwarnings('ignore')

In [10]:
# %%
os.chdir("..")  # Go up a directory
#print(os.getcwd())

In [12]:
from app import *

In [13]:
pip show streamlit

Name: streamlit
Version: 1.46.1
Summary: A faster way to build and share data apps
Home-page: https://streamlit.io
Author: Snowflake Inc
Author-email: hello@streamlit.io
License: Apache License 2.0
Location: C:\Users\hause\AppData\Roaming\Python\Python311\site-packages
Requires: altair, blinker, cachetools, click, gitpython, numpy, packaging, pandas, pillow, protobuf, pyarrow, pydeck, requests, tenacity, toml, tornado, typing-extensions, watchdog
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
os.system("streamlit run src/app.py")

2

In [16]:
import gradio as gr
import time

# Dummy retriever + generator for demonstration
def fake_retriever(question):
    # Returns top 2 source chunks (texts)
    return [
        "Source chunk 1: Customer complaints about login issues.",
        "Source chunk 2: Frequent crashes reported in app v2.0."
    ]

def fake_generator(prompt):
    # Simulate token streaming by yielding tokens slowly
    answer = "The main issues users report are login problems and app crashes."
    for i in range(len(answer)):
        yield answer[:i+1]
        time.sleep(0.05)

def rag_streaming(question):
    # Retrieve sources
    sources = fake_retriever(question)
    
    # Build prompt (skipped here for brevity)
    prompt = f"Context:\n{sources[0]}\n{sources[1]}\nQuestion: {question}\nAnswer:"
    
    # Stream generated answer tokens
    for partial_answer in fake_generator(prompt):
        yield partial_answer, sources

# Gradio interface

with gr.Blocks() as demo:
    gr.Markdown("## Financial Analyst Assistant with Sources and Streaming")
    
    question_input = gr.Textbox(label="Enter your question", lines=2)
    
    answer_output = gr.Textbox(label="AI-generated Answer", lines=6)
    
    source_output = gr.Textbox(label="Retrieved Sources", lines=6)
    
    ask_btn = gr.Button("Ask")
    clear_btn = gr.Button("Clear")
    
    # When ask_btn clicked, run rag_streaming and update answer + sources
    def ask_fn(question):
        # We return a generator, Gradio will stream the answer text and sources together
        gen = rag_streaming(question)
        # Initial empty answer and sources (sources will update on first yield)
        def generator():
            for answer, sources in gen:
                yield answer, "\n\n".join(sources)
        return generator()
    
    ask_btn.click(fn=ask_fn, inputs=question_input, outputs=[answer_output, source_output])
    
    def clear_fn():
        return "", ""
    clear_btn.click(fn=clear_fn, inputs=[], outputs=[answer_output, source_output])
    
demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
